<a href="https://colab.research.google.com/github/ykim71/thesis_related/blob/main/ner_pos_matching_wpreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
%cd drive/'MyDrive'/CrowdTangle/final_data_metrics

/content/drive/MyDrive/CrowdTangle/final_data_metrics




## pre-processing

In [14]:
# import words/phrases from NER and POS tagging
import pandas as pd

ner_pos = pd.read_excel('pos_bigrams_all_merged_NER4.xlsx')


In [15]:
import re

def bigrams(s):
    return [i for i in s if re.search(r'\s', i) ] 
def unigrams(s):
    return [i for i in s if not re.search(r'\s', i) ]
def hashtag(s):
    return [i for i in s if re.search(r'#', i) ]

def contain_the(s): # bigrams that make sense with 'the'
  to_remove = [' #', '# ', '.','\'s', "’s", " \'", "\'", "\'S", '“', '"', '’', ]
  p = re.compile('|'.join(map(re.escape, to_remove))) # escape to handle metachars
  s = [p.sub('', s) for s in s] # remove hashtags

  matchers = ['The ', 'the ']
  items = [i for i in s if any(xs in i for xs in matchers)]
  temp = [i for i in items if i.count(' ')==1]
  return list(set(temp))


In [16]:
# for Republican words/phrases

ner_rep_only = ner_pos.loc[(ner_pos['rep'] == 1)]

temp = ner_rep_only['text'].values.tolist()
ner_rep_only_list = list(set(temp))
print(len(ner_rep_only_list))

# bi-grams
rep_bi = list(set(bigrams(ner_rep_only_list)))

import re

to_remove = [' #', '# ', '.','\'s', "’s", " \'", "\'", "\'S", '“', '"', '’', ' Timeline', ' Photo', 'the ', 'The ']

p = re.compile('|'.join(map(re.escape, to_remove))) # escape to handle metachars

rep_bi = [p.sub('', s) for s in rep_bi] # remove hashtags

rep_bi = list(set(bigrams(rep_bi))) # select only bi-grams (hashtags will be added here)

# uni-grams

rep_uni = list(set(unigrams(ner_rep_only_list)))
print(len(rep_uni))
import re

to_remove = ['#', '.', '\'s', "’s", " \'", "\'", "\'S", '“', '"', '’']

p = re.compile('|'.join(map(re.escape, to_remove))) # escape to handle metachars

rep_uni = [p.sub('', s) for s in rep_uni] # remove some characters
rep_uni = list(set(rep_uni))
print(len(rep_uni))


# hashtags

import re

rep_hash = list(set(hashtag(ner_rep_only_list)))

to_remove = [' #', '# ', '#']
p = re.compile('|'.join(map(re.escape, to_remove))) # escape to handle metachars

rep_hash = [p.sub('', s) for s in rep_hash] # remove hashtags
rep_hash_non = list(set(rep_hash))

rep_hash = ['#' + s for s in rep_hash]
rep_hash = list(set(rep_hash))

# bigrams with the

rep_bi_the = contain_the(list(set(bigrams(ner_rep_only_list)))) 
print(len(rep_bi_the))

2712
185
181
1


In [17]:
rep_bi_hashtags = rep_bi + rep_hash + rep_bi_the
rep_uni_nonhashtags = rep_uni + rep_hash_non


In [18]:
# for Democratic words/phrases

ner_dem_only = ner_pos.loc[(ner_pos['dem'] == 1)]

temp = ner_dem_only['text'].values.tolist()
ner_dem_only_list = list(set(temp))
print(len(ner_dem_only_list))

# bi-grams
dem_bi = list(set(bigrams(ner_dem_only_list)))

import re

to_remove = [' #', '# ', '.','\'s', "’s", " \'", "\'", "\'S", '“', '"', '’', ' Timeline', ' Photo', 'the ', 'The ']

p = re.compile('|'.join(map(re.escape, to_remove))) # escape to handle metachars

dem_bi = [p.sub('', s) for s in dem_bi] # remove hashtags

dem_bi = list(set(bigrams(dem_bi))) # select only bi-grams (hashtags will be added here)


# uni-grams

dem_uni = list(set(unigrams(ner_dem_only_list)))
print(len(dem_uni))
import re

to_remove = ['#', '.', '\'s', "’s", " \'", "\'", "\'S", '“', '"', '’']

p = re.compile('|'.join(map(re.escape, to_remove))) # escape to handle metachars

dem_uni = [p.sub('', s) for s in dem_uni] # remove some characters
dem_uni = list(set(dem_uni))
print(len(dem_uni))

# hashtags

import re

dem_hash = list(set(hashtag(ner_dem_only_list)))

to_remove = [' #', '# ', '#']
p = re.compile('|'.join(map(re.escape, to_remove))) # escape to handle metachars

dem_hash = [p.sub('', s) for s in dem_hash] # remove hashtags
dem_hash_non = list(set(dem_hash))

dem_hash = ['#' + s for s in dem_hash]
dem_hash = list(set(dem_hash))


# bigrams with the

dem_bi_the = contain_the(list(set(bigrams(ner_dem_only_list)))) 


2533
175
173


In [19]:

dem_bi_hashtags = dem_bi + dem_hash + dem_bi_the
dem_uni_nonhashtags = dem_uni + dem_hash_non


In [21]:
# make lowercases for matching

temp_rep_lower_bi = list( set([x.lower() for x in rep_bi_hashtags] ))
temp_dem_lower_bi = list( set([x.lower() for x in dem_bi_hashtags] ))

temp_rep_lower_uni = list( set([x.lower() for x in rep_uni_nonhashtags]) )
temp_dem_lower_uni = list( set([x.lower() for x in dem_uni_nonhashtags]) )

# temp_rep_lower_bi_the = list( set([x.lower() for x in rep_bi_the] ))
# temp_dem_lower_bi_the = list( set([x.lower() for x in dem_bi_the] ))

In [22]:
print(len(temp_rep_lower_bi))
print(len(temp_dem_lower_bi))

print(len(temp_rep_lower_uni))
print(len(temp_dem_lower_uni))


2133
2059
245
236


## matching sample

In [ ]:
import pandas as pd

r_pages_st_sample = pd.read_csv("r_pages_st_sample.csv")
d_pages_st_sample = pd.read_csv("d_pages_st_sample.csv")


In [ ]:
d_pages_st_sample['leanings'] = 'democratic'
r_pages_st_sample['leanings'] = 'republican'

In [ ]:
sample_df_st = pd.concat([r_pages_st_sample, d_pages_st_sample], ignore_index=True, sort=False)


In [ ]:
sample_df_st['text_san'] = sample_df_st['text_san'].str.lower()


In [ ]:

sample_df_st['lower_ner_pos_rep_b'] = sample_df_st['text_san'].apply(lambda x: [i for i in temp_rep_lower_bi if i in x])
sample_df_st['lower_ner_pos_rep_u'] = sample_df_st['text_san'].apply(lambda x: set.intersection(set(x.split(' ')), temp_rep_lower_uni))

sample_df_st['lower_ner_pos_dem_b'] = sample_df_st['text_san'].apply(lambda x: [i for i in temp_dem_lower_bi if i in x])
sample_df_st['lower_ner_pos_dem_u'] = sample_df_st['text_san'].apply(lambda x: set.intersection(set(x.split(' ')), temp_dem_lower_uni))


In [ ]:
sample_df_st.to_csv('sample_df_st_lower.csv')

## matching

In [ ]:
# for entire posts

def basic_sanitize(text):
    
    text = re.sub(r'[\'’]',' ', text) #&
    text = re.sub(r'[^\w\s#&-]','', text) #&
    text = text.lower()
    
    return text

### for Republican posts

In [ ]:
import pickle

with open('r_pages_ner_pos.pickle', 'rb') as handle:
    r_pages_st = pickle.load(handle)

In [ ]:
r_pages_st['text_san'] = r_pages_st['text'].apply(basic_sanitize)


In [ ]:
r_pages_st['text_san'].sample(3)

3030277    waking up   wwwmainwashedcom timeline photos l...
6832155     trump signs order to improve mental health re...
5330745    look what the intolerant left did to our frien...
Name: text_san, dtype: object

In [ ]:
print(len(temp_rep_lower_bi))
print(len(temp_dem_lower_bi))

print(len(temp_rep_lower_uni))
print(len(temp_dem_lower_uni))

2132
2055
246
236


In [ ]:
# matching

r_pages_st['ner_pos_rep_b'] = r_pages_st['text_san'].apply(lambda x: [i for i in temp_rep_lower_bi if i in x])
r_pages_st['ner_pos_rep_u'] = r_pages_st['text_san'].apply(lambda x: set.intersection(set(x.split(' ')), temp_rep_lower_uni))


In [ ]:
r_pages_st['ner_pos_dem_b'] = r_pages_st['text_san'].apply(lambda x: [i for i in temp_dem_lower_bi if i in x])
r_pages_st['ner_pos_dem_u'] = r_pages_st['text_san'].apply(lambda x: set.intersection(set(x.split(' ')), temp_dem_lower_uni))


In [ ]:
# saving results

r_pages_st5 = r_pages_st[['quarter','insult','sentistr','ner_pos_rep_u','ner_pos_rep_b','ner_pos_dem_u','ner_pos_dem_b']]

import pickle

with open('r_pages_ner_pos_3rd_lowercase.pickle', 'wb') as handle:
    pickle.dump(r_pages_st5, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [ ]:

# print('in_group_np')
# print(r_pages_st['in_group_np'].value_counts())

# print('out_group_np')
# print(r_pages_st['out_group_np'].value_counts())

# print('no_entity_np')
# print(r_pages_st['no_entity_np'].value_counts())


# temp  = r_pages_st.groupby(['quarter', 'out_group_np'])['sentistr_std'].mean().reset_index()
# temp.to_csv('pos_ner_lower/r_out_group_np_senti.csv', encoding='utf-8-sig')

# temp  = r_pages_st.groupby(['quarter', 'out_group_np'])['insult_std'].mean().reset_index()
# temp.to_csv('pos_ner_lower/r_out_group_np_inciv.csv', encoding='utf-8-sig')

# temp  = r_pages_st.groupby(['quarter', 'in_group_np'])['sentistr_std'].mean().reset_index()
# temp.to_csv('pos_ner_lower/r_in_group_np_senti.csv', encoding='utf-8-sig')

# temp  = r_pages_st.groupby(['quarter', 'in_group_np'])['insult_std'].mean().reset_index()
# temp.to_csv('pos_ner_lower/r_in_group_np_inciv.csv', encoding='utf-8-sig')

# temp  = r_pages_st.groupby(['quarter', 'no_entity_np'])['sentistr_std'].mean().reset_index()
# temp.to_csv('pos_ner_lower/r_no_entity_np_senti.csv', encoding='utf-8-sig')

# temp  = r_pages_st.groupby(['quarter', 'no_entity_np'])['insult_std'].mean().reset_index()
# temp.to_csv('pos_ner_lower/r_no_entity_np_inciv.csv', encoding='utf-8-sig')




in_group_np
non_mentioned    4352731
mentioned        2997907
Name: in_group_np, dtype: int64
out_group_np
non_mentioned    4730356
mentioned        2620282
Name: out_group_np, dtype: int64
no_entity_np
mentioned    4238245
none         3112393
Name: no_entity_np, dtype: int64


In [ ]:
# manual checking

r_pages_st_sample = r_pages_st.sample(1000)
r_pages_st_sample.to_csv('r_pages_st_sample_lower.csv', encoding='utf-8-sig')


### for Democratic posts

In [13]:
import pickle

with open('d_pages_ner_pos.pickle', 'rb') as handle:
    d_pages_st = pickle.load(handle)

In [24]:
d_pages_st['text_san'] = d_pages_st['text'].apply(basic_sanitize)


In [25]:
d_pages_st['text_san'].sample(3)

5164760     trump seen golfing at his virginia club over ...
1841491     uniteblue we cannot stay silent as this lawle...
2170569    what is fuckabee afraid the nsa heard surely h...
Name: text_san, dtype: object

In [26]:

d_pages_st['ner_pos_rep_b'] = d_pages_st['text_san'].apply(lambda x: [i for i in temp_rep_lower_bi if i in x])

d_pages_st['ner_pos_rep_u'] = d_pages_st['text_san'].apply(lambda x: set.intersection(set(x.split(' ')), temp_rep_lower_uni))


In [27]:
d_pages_st['ner_pos_dem_b'] = d_pages_st['text_san'].apply(lambda x: [i for i in temp_dem_lower_bi if i in x])

d_pages_st['ner_pos_dem_u'] = d_pages_st['text_san'].apply(lambda x: set.intersection(set(x.split(' ')), temp_dem_lower_uni))



In [28]:
d_pages_st5 = d_pages_st[['quarter','insult','sentistr','ner_pos_rep_u','ner_pos_rep_b','ner_pos_dem_u','ner_pos_dem_b']]


In [29]:

import pickle

with open('d_pages_ner_pos_3rd_lowercase.pickle', 'wb') as handle:
    pickle.dump(d_pages_st5, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [21]:

# print('in_group_np')
# print(d_pages_st['in_group_np'].value_counts())

# print('out_group_np')
# print(d_pages_st['out_group_np'].value_counts())

# print('no_entity_np')
# print(d_pages_st['no_entity_np'].value_counts())


# temp  = d_pages_st.groupby(['quarter', 'out_group_np'])['sentistr_std'].mean().reset_index()
# temp.to_csv('pos_ner_lower/d_out_group_np_senti.csv', encoding='utf-8-sig')

# temp  = d_pages_st.groupby(['quarter', 'out_group_np'])['insult_std'].mean().reset_index()
# temp.to_csv('pos_ner_lower/d_out_group_np_inciv.csv', encoding='utf-8-sig')

# temp  = d_pages_st.groupby(['quarter', 'in_group_np'])['sentistr_std'].mean().reset_index()
# temp.to_csv('pos_ner_lower/d_in_group_np_senti.csv', encoding='utf-8-sig')

# temp  = d_pages_st.groupby(['quarter', 'in_group_np'])['insult_std'].mean().reset_index()
# temp.to_csv('pos_ner_lower/d_in_group_np_inciv.csv', encoding='utf-8-sig')

# temp  = d_pages_st.groupby(['quarter', 'no_entity_np'])['sentistr_std'].mean().reset_index()
# temp.to_csv('pos_ner_lower/d_no_entity_np_senti.csv', encoding='utf-8-sig')

# temp  = d_pages_st.groupby(['quarter', 'no_entity_np'])['insult_std'].mean().reset_index()
# temp.to_csv('pos_ner_lower/d_no_entity_np_inciv.csv', encoding='utf-8-sig')


in_group_np
non_mentioned    4373991
mentioned        2436033
Name: in_group_np, dtype: int64
out_group_np
non_mentioned    3643522
mentioned        3166502
Name: out_group_np, dtype: int64
no_entity_np
mentioned    4299352
none         2510672
Name: no_entity_np, dtype: int64


In [ ]:
# manual checking
d_pages_st_sample = d_pages_st.sample(1000)
d_pages_st_sample.to_csv('d_pages_st_sample_lower.csv', encoding='utf-8-sig')


## merge results from both leanings

In [3]:
import pickle

with open('r_pages_ner_pos_3rd_lowercase.pickle', 'rb') as handle:
    r_pages_st = pickle.load(handle)

In [4]:

with open('d_pages_ner_pos_3rd_lowercase.pickle', 'rb') as handle:
    d_pages_st = pickle.load(handle)

In [5]:
r_pages_st['leanings'] = "republican"
d_pages_st['leanings'] = "democratic"

In [6]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

d_pages_st[['sentistr_std', 'insult_std']] = scaler.fit_transform(d_pages_st[['sentistr', 'insult']])

r_pages_st[['sentistr_std', 'insult_std']] = scaler.fit_transform(r_pages_st[['sentistr', 'insult']])


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:991: RuntimeWarning: invalid value encountered in subtract
  temp = X - T
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:993: RuntimeWarning: invalid value encountered in subtract
  X -= self.mean_
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:991: RuntimeWarning: invalid value encountered in subtract
  temp = X - T
/usr/local/lib/python3.7/dist-package

In [ ]:

import numpy as np

d_pages_st['ner_pos_dem_b_count'] = d_pages_st['ner_pos_dem_b'].str.len()
d_pages_st['ner_pos_dem_u_count'] = d_pages_st['ner_pos_dem_u'].str.len()

d_pages_st['ner_pos_rep_b_count'] = d_pages_st['ner_pos_rep_b'].str.len()
d_pages_st['ner_pos_rep_u_count'] = d_pages_st['ner_pos_rep_u'].str.len()


d_pages_st['out_group_np'] = np.where(( (d_pages_st.ner_pos_rep_u_count >= 1) | (d_pages_st.ner_pos_rep_b_count >= 1) ), 'mentioned', 'non_mentioned')
d_pages_st['in_group_np'] = np.where(( (d_pages_st.ner_pos_dem_u_count >= 1) | (d_pages_st.ner_pos_dem_b_count >= 1) ), 'mentioned', 'non_mentioned')
d_pages_st['no_entity_np'] = np.where(((d_pages_st.in_group_np == 'non_mentioned') & (d_pages_st.out_group_np == 'non_mentioned')), 'none', 'mentioned')


In [ ]:
import numpy as np

r_pages_st['ner_pos_dem_b_count'] = r_pages_st['ner_pos_dem_b'].str.len()
r_pages_st['ner_pos_dem_u_count'] = r_pages_st['ner_pos_dem_u'].str.len()

r_pages_st['ner_pos_rep_b_count'] = r_pages_st['ner_pos_rep_b'].str.len()
r_pages_st['ner_pos_rep_u_count'] = r_pages_st['ner_pos_rep_u'].str.len()


r_pages_st['in_group_np'] = np.where(( (r_pages_st.ner_pos_rep_u_count >= 1) | (r_pages_st.ner_pos_rep_b_count >= 1) ), 'mentioned', 'non_mentioned')
r_pages_st['out_group_np'] = np.where(( (r_pages_st.ner_pos_dem_u_count >= 1) | (r_pages_st.ner_pos_dem_b_count >= 1) ), 'mentioned', 'non_mentioned')
r_pages_st['no_entity_np'] = np.where(((r_pages_st.in_group_np == 'non_mentioned') & (r_pages_st.out_group_np == 'non_mentioned')), 'none', 'mentioned')


In [7]:
d_pages_st.columns

Index(['quarter', 'insult', 'sentistr', 'ner_pos_rep_u', 'ner_pos_rep_b',
       'ner_pos_dem_u', 'ner_pos_dem_b', 'leanings', 'sentistr_std',
       'insult_std'],
      dtype='object')

In [8]:
r_pages_st.columns

Index(['quarter', 'insult', 'sentistr', 'ner_pos_rep_u', 'ner_pos_rep_b',
       'ner_pos_dem_u', 'ner_pos_dem_b', 'leanings', 'sentistr_std',
       'insult_std'],
      dtype='object')

In [11]:
import pandas as pd

pages_df_st = pd.concat([r_pages_st, d_pages_st], ignore_index=True, sort=False)


In [12]:
# aggregated by all posts

temp  = pages_df_st.groupby(['quarter', 'out_group_np'])['sentistr_std'].mean().reset_index()
temp.to_csv('pos_ner_lower/all_np_out_group_senti_std.csv', encoding='utf-8-sig')


temp  = pages_df_st.groupby(['quarter', 'in_group_np'])['sentistr_std'].mean().reset_index()
temp.to_csv('pos_ner_lower/all_np_in_group_senti_std.csv', encoding='utf-8-sig')


temp  = pages_df_st.groupby(['quarter', 'no_entity_np'])['sentistr_std'].mean().reset_index()
temp.to_csv('pos_ner_lower/all_np_no_entities_senti_std.csv', encoding='utf-8-sig')


temp  = pages_df_st.groupby(['quarter', 'out_group_np'])['insult_std'].mean().reset_index()
temp.to_csv('pos_ner_lower/all_np_out_group_insult_std.csv', encoding='utf-8-sig')


temp  = pages_df_st.groupby(['quarter', 'in_group_np'])['insult_std'].mean().reset_index()
temp.to_csv('pos_ner_lower/all_np_in_group_insult_std.csv', encoding='utf-8-sig')


temp  = pages_df_st.groupby(['quarter', 'no_entity_np'])['insult_std'].mean().reset_index()
temp.to_csv('pos_ner_lower/all_np_no_entities_insult_std.csv', encoding='utf-8-sig')


In [14]:
# aggregated by all posts + by leanings

temp  = pages_df_st.groupby(['quarter', 'out_group_np', 'leanings'])['sentistr_std'].mean().reset_index()
temp.to_csv('pos_ner_lower/leanings_np_out_group_senti_std.csv', encoding='utf-8-sig')


temp  = pages_df_st.groupby(['quarter', 'in_group_np', 'leanings'])['sentistr_std'].mean().reset_index()
temp.to_csv('pos_ner_lower/leanings_np_in_group_senti_std.csv', encoding='utf-8-sig')


temp  = pages_df_st.groupby(['quarter', 'no_entity_np', 'leanings'])['sentistr_std'].mean().reset_index()
temp.to_csv('pos_ner_lower/leanings_np_no_entities_senti_std.csv', encoding='utf-8-sig')


temp  = pages_df_st.groupby(['quarter', 'out_group_np', 'leanings'])['insult_std'].mean().reset_index()
temp.to_csv('pos_ner_lower/leanings_np_out_group_insult_std.csv', encoding='utf-8-sig')


temp  = pages_df_st.groupby(['quarter', 'in_group_np', 'leanings'])['insult_std'].mean().reset_index()
temp.to_csv('pos_ner_lower/leanings_np_in_group_insult_std.csv', encoding='utf-8-sig')


temp  = pages_df_st.groupby(['quarter', 'no_entity_np', 'leanings'])['insult_std'].mean().reset_index()
temp.to_csv('pos_ner_lower/leanings_np_no_entities_insult_std.csv', encoding='utf-8-sig')
